In [11]:
import duckdb
import pandas as pd
import geopandas as gpd

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 10

pd.set_option("display.precision", 2)

con = duckdb.connect()

def plot_gdf(res, col):
    # conver the result to a GeoDataFrame, converting from WKT to geometry
    res_gdf = gpd.GeoDataFrame(res, geometry=gpd.GeoSeries.from_wkt(res['geom']))
    # Plot the result, coloring by the 'vendor_id' column
    res_gdf.plot(column=col, legend=True)

%reload_ext sql
%sql con --alias duckdb

In [12]:
%%sql

-- Install and load the DuckDB "spatial" extension

INSTALL spatial;
LOAD spatial

,Success


In [3]:
%%sql

-- Create a table with a spatial GEOMETRY column

CREATE TABLE spatial_table (id INT, geom GEOMETRY);

-- Insert a point created with the ST_Point function
INSERT INTO spatial_table VALUES (1, ST_Point(32, 42));

-- Insert a line created with the ST_GeomFromText function
INSERT INTO spatial_table VALUES (2, ST_GeomFromText('LINESTRING (0 0, 1 1, 2 2)'));

-- Select all rows from the table
SELECT id, ST_AsText(geom) FROM spatial_table;

,id,st_astext(geom)
0,1,POINT (32 42)
1,2,"LINESTRING (0 0, 1 1, 2 2)"


In [36]:
%%sql

-- Lets import the taxi trips again, this time creating points from the dropoff coordinates
CREATE TABLE taxi_trips_points AS
SELECT 
    vendor_id, 
    trip_distance, 
    ST_Point(dropoff_latitude, dropoff_longitude) AS geom
FROM read_csv('./taxi_2010_01_1m.csv');

RuntimeError: Catalog Error: Table with name "taxi_trips_points" already exists!
If you need help solving this issue, send us a message: https://ploomber.io/community


In [27]:
%%sql

-- How many trips ended within 100 meters of the Empire State Building?
-- 100 meters is approximately 0.0009 degrees (at the equator)

SELECT COUNT(*)
FROM taxi_trips_points
WHERE ST_Distance(geom, ST_Point(40.748817, -73.985428)) < (0.0009);


,count_star()
0,944


In [29]:
%%sql

-- Import the taxi zones as geometries using the ST_Read table function
CREATE OR REPLACE TABLE taxi_zones AS SELECT * FROM ST_Read('taxi_zones/taxi_zones.shp');

-- Display the first 5 rows
SELECT borough, zone, ST_AsText(geom) FROM taxi_zones LIMIT 5;


,borough,zone,st_astext(geom)
0,EWR,Newark Airport,POLYGON ((933100.9183527103 192536.08569720192...
1,Queens,Jamaica Bay,MULTIPOLYGON (((1033269.2435912937 172126.0078...
2,Bronx,Allerton/Pelham Gardens,POLYGON ((1026308.7695066631 256767.6975403726...
3,Manhattan,Alphabet City,POLYGON ((992073.4667968601 203714.07598876953...
4,Staten Island,Arden Heights,POLYGON ((935843.3104932606 144283.33585065603...


## Coordinate Systems (back to the slides)

In [55]:
%%sql

-- The taxi zones are in the EPSG:2263 projection. Let's convert them to EPSG:4326
-- using the ST_Transform function, taking the source and target EPSG codes as arguments

CREATE OR REPLACE TABLE taxi_zones_4326 AS 
SELECT
    borough, 
    zone,
    ST_Transform(geom, 'EPSG:2263', 'EPSG:4326') AS geom
FROM taxi_zones;

,Count
0,263


In [59]:
%%sql

SELECT borough, zone, ST_AsText(geom) FROM taxi_zones_4326 LIMIT 5;


,borough,zone,st_astext(geom)
0,EWR,Newark Airport,POLYGON ((40.694996000000096 -74.1844529999999...
1,Queens,Jamaica Bay,MULTIPOLYGON (((40.63898704717692 -73.82337597...
2,Bronx,Allerton/Pelham Gardens,"POLYGON ((40.8713422340001 -73.84792614099985,..."
3,Manhattan,Alphabet City,POLYGON ((40.72582128133726 -73.97177410965318...
4,Staten Island,Arden Heights,POLYGON ((40.56256808600009 -74.17421738099989...


In [57]:
%%sql

-- Now that both tables are in the same projection, lets use a spatial JOIN 
-- to count the number of taxi pickups in each zone!

SELECT 
    COUNT(*) as dropoff_count, 
    taxi_zones_4326.borough
FROM 
    taxi_zones_4326
JOIN 
    taxi_trips_points
ON 
    ST_Contains(taxi_zones_4326.geom, taxi_trips_points.geom)
GROUP BY 
    taxi_zones_4326.borough
ORDER BY
    dropoff_count DESC;

,count_star(),borough
0,4605,Bronx
1,883532,Manhattan
2,251,Staten Island
3,1004,EWR
4,41989,Brooklyn
5,44583,Queens
